# Translating simple analyses from Jason Zevin with some extra plots

### Environment

In [ ]:
%load_ext lab_black
import pandas as pd
import altair as alt

### Ingest, tidy

In [ ]:
df = pd.read_csv("1250_sims.csv", index_col=0)

# Rename variables to use old codes
# Should avoid variable with ".", since it will block some convienent function in pandas
df.rename(
    columns={
        "ID": "code_name",
        "Trial.Scaled": "epoch",
        "Hidden": "hidden_units",
        "PhoHid": "cleanup_units",
        "Pnoise": "p_noise",
        "Epsilon": "learning_rate",
        "Type": "cond",
        "Measure": "measure",
        "Score": "score",
        "Freq": "cond_freq",
        "Cons": "cond_cons",
    },
    inplace=True,
)

# h-params that varies
variates = ["hidden_units", "cleanup_units", "p_noise", "learning_rate"]

### Create H-params settings table and count model per setting

In [ ]:
settings = df[["code_name"] + variates].pivot_table(index="code_name")
settings["code_name"] = settings.index
settings["learning_rate"].round(4)

count_settings = settings.pivot_table(
    index=variates, aggfunc="count", values="code_name",
)
count_settings.reset_index(inplace=True)
count_settings.rename(columns={"code_name": "n"}, inplace=True)

### How many model per cell? 2

In [ ]:
plot_count = (
    alt.Chart(count_settings)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="cleanup_units:O",
        color="n:O",
        tooltip=variates + ["n"],
    )
    .properties(title="Model counts")
)

plot_count.save("count_model.html")

# Strain related plots

## Histogram of overall accuracy

In [ ]:
df_last_epoch = df.loc[df.epoch == df.epoch.max()]
df_last_epoch_strain = df_last_epoch.loc[df_last_epoch.cond_cons != "NW"]

In [ ]:
hist_strain = (
    alt.Chart(df_last_epoch_strain)
    .mark_bar()
    .encode(alt.X("score:Q", bin=True), y="count()",)
    .properties(title="Over all accuracy in Strain (all conditions)")
)

hist_strain.save("Strain_histogram.html")

## Histogram of accuracy by condition

In [ ]:
hist_strain_cond = hist_strain.encode(color="cond:N", column="cond:N").properties(
    title="Accuracy in Strain by conditions", width=100, height=50
)
hist_strain_cond.save("Strain_histogram_cond.html")

### descriptives

In [ ]:
print("Descriptives on Grand mean accuracy in Strain:")
df_last_epoch_strain.score.describe()

In [ ]:
print("Descriptives on Mean accuracy by condition in Strain")
df_last_epoch_strain.groupby("cond").score.describe()

### Accuracy in the last epoch

In [ ]:
plot_meanacc_bycond = (
    alt.Chart(df_last_epoch_strain)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        column="learning_rate:O",
        row="cond:N",
        color=alt.Color(
            "mean(score)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        tooltip=["mean(score)"],
    )
    .properties(title="Mean accuracy in each condition")
)

plot_meanacc_bycond.save("Strain_acc_bycond.html")

## Significant interactions (refers to R for details):
- Hidden x Pnoise x Epsilon
- Frequency x Consistency x Pnoise x Hidden
- Frequency x Consistency x Pnoise x Epsilon

### Overall accuracy (Hidden x Pnoise x Epsilon)
- the platform I am using have some trouble in plotting large dataset, it is easier for me to save the plot instead of plotting it in the notebook

In [ ]:
plot_meanacc_strain = (
    alt.Chart(df_last_epoch_strain)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        column="learning_rate:O",
        color=alt.Color(
            "mean(score)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        tooltip=["mean(score)"],
    )
    .properties(title="Mean accuracy (all conditions in Strain)")
)

plot_meanacc_strain.save("Strain_hparam_effect_on_acc.html")

### Frequency by Consistency interaction (facet by: hidden unit and noise)

In [ ]:
fxc_hn = (
    alt.Chart(df_last_epoch_strain)
    .mark_boxplot()
    .encode(x="cond:O", y="score:Q", column="p_noise", row="hidden_units")
    .properties(width=100, height=80)
)

fxc_hn.save("Strain_interaction_hidden_pnoise_boxplot.html")

### Frequency by Consistency interaction (facet by: Learning rate and noise)

In [ ]:
fxc_ln = fxc_hn.encode(row="learning_rate")  # Recycle last plot (fxc_hn) for new facet
fxc_ln.save("Strain_interaction_lr_pnoise_boxplot.html")

- the boxplots are still quite hard to read... maybe aggregating interaction can help
- Define interact: (LF_CON - LF_INC) - (HF_CON - HF_INC)
- (Maybe later) Probably adding SD is better... I meant plotting the t-value.

In [ ]:
df_strain_int_eff = df_last_epoch_strain[variates + ["cond", "score", "code_name"]]

df_strain_int_eff = df_strain_int_eff.pivot_table(
    index=variates + ["code_name"], columns="cond"
).reset_index()

df_strain_int_eff.columns = df_strain_int_eff.columns = [
    "".join(c).strip() for c in df_strain_int_eff.columns.values
]

df_strain_int_eff.rename(
    columns={
        "scoreHF_CON": "HF_CON",
        "scoreHF_INC": "HF_INC",
        "scoreLF_CON": "LF_CON",
        "scoreLF_INC": "LF_INC",
    },
    inplace=True,
)

df_strain_int_eff["fxc"] = (
    df_strain_int_eff.LF_CON
    - df_strain_int_eff.LF_INC
    - df_strain_int_eff.HF_CON
    + df_strain_int_eff.HF_INC
)

### Interaction effect heatmap

In [ ]:
plot_interaction_ph = (
    alt.Chart(df_strain_int_eff)
    .mark_rect()
    .encode(
        x="p_noise:O", y="hidden_units:O", color="mean(fxc)", tooltip=["mean(fxc)"],
    )
    .properties(
        title="Interaction heatmap for visualising: (FREQ x CON) x PNOISE x Hidden"
    )
)

plot_interaction_ph.save("Strain_interaction_pnoise_hidden_heatmap.html")

In [ ]:
plot_interaction_pl = plot_interaction_ph.encode(y="learning_rate:O").properties(
    title="Interaction heatmap for visualising: (FREQ x CON) x PNOISE x Epsilon"
)

plot_interaction_pl.save("Strain_interaction_pnoise_lr_heatmap.html")

# Grain related plots

In [ ]:
df_last_epoch_grain = df_last_epoch.loc[
    (df_last_epoch.cond.isin(["NW_UN", "NW_AMB"]))
    & (df_last_epoch.measure == "Accuracy")
]

hist_grain = (
    alt.Chart(df_last_epoch_grain)
    .mark_bar()
    .encode(alt.X("score:Q", bin=True), y="count()",)
    .properties(title="Over all accuracy in Grain (all conditions)")
)

hist_grain.save("Grain_histogram.html")

In [ ]:
hist_grain_cond = hist_grain.encode(color="cond:N", column="cond:N").properties(
    title="Accuracy in Grain by conditions", width=100, height=50
)
hist_grain_cond.save("Grain_histogram_cond.html")

In [ ]:
print("Descriptives on Grand mean accuracy in Grain:")
df_last_epoch_grain.score.describe()

In [ ]:
print("Descriptives on Mean accuracy by condition in Grain")
df_last_epoch_grain.groupby("cond").score.describe()

### Grain by condition plot at last epoch

In [ ]:
plot_meanacc_grain = (
    alt.Chart(df_last_epoch_grain)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        column="learning_rate:O",
        row="cond:N",
        color=alt.Color(
            "mean(score)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        tooltip=["mean(score)"],
    )
    .properties(title="Mean accuracy in each condition in Grain")
)

plot_meanacc_grain.save("Grain_acc_bycond.html")

## Significant interactions (See R for details)
- Pnoise x Hidden X Epsilon
- Type x Hidden x Epsilon x PhoHid
- Type x Pnoise x PhoHid
- Type x Pnoise x Epsilon

### Pnoise x Hidden X Epsilon

In [ ]:
plot_meanacc = (
    alt.Chart(df_last_epoch_grain)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        column="learning_rate:O",
        color=alt.Color(
            "mean(score)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        tooltip=["mean(score)"],
    )
    .properties(title="Mean accuracy (all conditions in Strain)")
)

plot_meanacc.save("Grain_hparma_effect_on_acc.html")

### Type x h-params interactions
- Since there are 4 ways interactions, to reduce the plot number, aggregate condition main effect = NW_UN - NW_AMB

In [ ]:
df_grain_cond_eff = df_last_epoch_grain[variates + ["cond", "score", "code_name"]]

df_grain_cond_eff = df_grain_cond_eff.pivot_table(
    index=variates + ["code_name"], columns="cond"
).reset_index()

df_grain_cond_eff.columns = df_grain_cond_eff.columns = [
    "".join(c).strip() for c in df_grain_cond_eff.columns.values
]

df_grain_cond_eff.rename(
    columns={"scoreNW_AMB": "ambiguous", "scoreNW_UN": "unambiguous"}, inplace=True
)

df_grain_cond_eff["cond_effect"] = (
    df_grain_cond_eff.unambiguous - df_grain_cond_eff.ambiguous
)

In [ ]:
plot_effect = (
    alt.Chart(df_grain_cond_eff)
    .mark_rect()
    .encode(
        x="cleanup_units:O",
        y="hidden_units:O",
        column="learning_rate:O",
        color="mean(cond_effect)",
        tooltip=["mean(cond_effect)"],
    )
    .properties(
        title="Mean condition effect (UN - AMB) for visualising: Type x Hidden x Epsilon x PhoHid"
    )
)

plot_effect.save("Grain_interaction_lr_hidden_cleanup.html")

Hard to tell the interaction... maybe higher condition effect in more cleanup, fewer hidden, and larger epsilon...

### Type x Pnoise x PhoHid

In [ ]:
# Original Boxplot
interaction_hc = (
    alt.Chart(df_last_epoch_grain)
    .mark_boxplot()
    .encode(x="cond:O", y="score:Q", row="p_noise", column="cleanup_units")
    .properties(width=100, height=80)
)

interaction_hc.save("Grain_interaction_pnoise_cleanup_boxplot.html")

In [ ]:
# Easier to read heatmap
plot_effect = (
    alt.Chart(df_grain_cond_eff)
    .mark_rect()
    .encode(
        x="cleanup_units:O",
        y="p_noise:O",
        color="mean(cond_effect)",
        tooltip=["mean(cond_effect)"],
    )
)

plot_effect.save("Grain_interaction_pnoise_cleanup_heatmap.html")

### Type x Pnoise x Epsilon

In [ ]:
# Original Boxplot
interaction_pe = interaction_hc.encode(row="p_noise", column="learning_rate")
interaction_pe.save("Grain_interaction_pnoise_lr_boxplot.html")

In [ ]:
# Easier to read heatmap
plot_effect.encode(x="learning_rate:O").save("Grain_interaction_pnoise_lr_heatmap.html")

There are some response type plots but skipping it for now...

# Some new plots

### Book chapter figure 2

Create ploting dataframe
- Only Accuracy (since NW has other measures)
- Only HF_INC and NW_UN (to match original plot axis selection)
- H-param cell aggregation (n=2 per cell)

In [ ]:
df_wnw = df.loc[
    (df.cond.isin(["HF_INC", "NW_UN"])) & (df.measure == "Accuracy"),
    variates + ["code_name", "epoch", "cond", "score"],
]

df_wnw = df_wnw.pivot_table(index=variates + ["epoch"], columns="cond").reset_index()
df_wnw.columns = df_wnw.columns = ["".join(c).strip() for c in df_wnw.columns.values]

df_wnw.rename(
    columns={
        "scoreHF_INC": "word_acc",
        "scoreNW_UN": "nonword_acc",
        "code_nameHF_INC": "code_name",
    },
    inplace=True,
)

df_wnw.drop(columns="code_nameNW_UN", inplace=True)
df_wnw["word_advantage"] = df_wnw.word_acc - df_wnw.nonword_acc

Plotting Word (HF_INC) vs. Nonword (NW_UN) accuracy in all epoch

In [ ]:
alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

base = (
    alt.Chart(df_wnw)
    .mark_point()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        color=alt.Color(
            "epoch", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        opacity=alt.value(0.1),
        tooltip=["epoch", "word_acc", "nonword_acc"],
    )
)

diagonal = (
    alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
    .mark_line(color="black")
    .encode(x="x", y="y")
)


fig2 = (diagonal + base).properties(
    title="Word (HF_INC) vs. Nonword (NW_UN) accuracy in all epoch"
)
fig2.save("fig2.html")

### Heatmap by epoch
- Since no significant interaction with cleanup plot it as a selector to double check

In [ ]:
sel_cleanup = alt.selection(
    type="single",
    on="click",
    fields=["cleanup_units"],
    bind=alt.binding_radio(options=df_wnw.cleanup_units.unique(), name="Cleanup: "),
)

hm_base = (
    alt.Chart(df_wnw)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="epoch:O",
        tooltip=["word_acc", "nonword_acc", "word_advantage"] + variates,
    )
    .add_selection(sel_cleanup)
    .transform_filter(sel_cleanup)
)

# Word
hm_word = hm_base.encode(
    color=alt.Color(
        "mean(word_acc)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
    )
).properties(title="Word accuracy")

# Nonword
hm_nonword = hm_base.encode(
    color=alt.Color(
        "mean(nonword_acc)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
    )
).properties(title="Nonword accuracy")

# Word - Nonword
hm_wordadvantage = hm_base.encode(
    color=alt.Color(
        "mean(word_advantage)",
        scale=alt.Scale(scheme="redyellowgreen", domain=(-0.3, 0.3)),
    )
).properties(title="Word advantage")


hm_word.save("heatmap_word_with_cleanup.html")
hm_nonword.save("heatmap_nonword_with_cleanup.html")
hm_wordadvantage.save("heatmap_wordadvantage_with_cleanup.html")

### Seems save to aggregate cleanup

In [ ]:
hm_base_nc = (
    alt.Chart(df_wnw)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="epoch:O",
        tooltip=["word_acc", "nonword_acc", "word_advantage"],
    )
)

# Word
hm_word = hm_base_nc.encode(
    color=alt.Color(
        "mean(word_acc)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
    )
).properties(title="Word accuracy")

# Nonword
hm_nonword = hm_base_nc.encode(
    color=alt.Color(
        "mean(nonword_acc)", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
    )
).properties(title="Nonword accuracy")

# Word - Nonword
hm_wordadvantage = hm_base_nc.encode(
    color=alt.Color(
        "mean(word_advantage)",
        scale=alt.Scale(scheme="redyellowgreen", domain=(-0.3, 0.3)),
    )
).properties(title="Word advantage")


hm_word.save("heatmap_word.html")
hm_nonword.save("heatmap_nonword.html")
hm_wordadvantage.save("heatmap_wordadvantage.html")

## Interactive dashboard

In [ ]:
alt.data_transformers.disable_max_rows()

def main_dashboard(df):

    sel_run = alt.selection(type="multi", on="click", fields=["code_name"])

    # df for overview
    df_ov = df[df.epoch == df.epoch.max()]

    # Shared master over-view heatmap
    overview = (
        alt.Chart(df_ov)
        .mark_rect()
        .encode(
            x="p_noise:O",
            y="hidden_units:O",
            row="learning_rate:O",
            column="cleanup_units:O",
            color=alt.Color(
                "word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
            ),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
            tooltip=["word_acc", "nonword_acc", "word_advantage"],
        )
        .add_selection(sel_run)
        .properties(title="Word accuracy at the end of training")
    )

    wnw_mdf = df.melt(
        id_vars=["code_name", "epoch"],
        value_vars=["word_acc", "nonword_acc"],
        var_name="wnw",
        value_name="acc",
    )

    # Developmental plot
    plot_epoch = (
        alt.Chart(wnw_mdf)
        .mark_point(size=80)
        .encode(
            y=alt.Y("acc:Q", scale=alt.Scale(domain=(0, 1))),
            x="epoch:Q",
            color=alt.Color("code_name:N", legend=None),
            shape="wnw:N",
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["code_name", "epoch", "acc"],
        )
        .add_selection(sel_run)
        .properties(title="Plot word and nonword accuracy by epoch")
    )

    # Word against Nonword
    wnw_line = (
        alt.Chart(df)
        .mark_line()
        .encode(
            y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
            x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
            color=alt.Color("code_name:N", legend=None),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
        )
    )

    wnw_point = wnw_line.mark_point().encode(
        color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen"))
    )

    diagonal = (
        alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
        .mark_line(color="black")
        .encode(x="x", y="y")
    )

    wnw = diagonal + wnw_line + wnw_point

    wnw_interactive = wnw.add_selection(sel_run).properties(
        title="Word vs. Nonword accuracy at final time step"
    )

    ### Mini heatmap ###

    mini_wadv = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            x="epoch:O",
            color=alt.Color(
                "word_advantage:Q",
                scale=alt.Scale(scheme="redyellowgreen", domain=(-0.3, 0.3)),
            ),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["word_acc", "nonword_acc", "word_advantage"] + variates,
        )
        .properties(title="Word - Nonword")
    )

    return (overview | (plot_epoch & mini_wadv) | wnw_interactive).resolve_scale(
        y="independent", color="independent", shape="independent"
    )


main_dashboard(df_wnw).save("dashboard.html")